In [ ]:
import json

# Все категории с координатами
categories = {
    "Достопримечательности": [
        ("Гора Змейка", 44.169306, 43.098203),
        ("Привокзальная площадь", 44.212209, 43.140721),
        ("Площадь Победы", 44.201252, 43.116690),
        ("Собор Покрова Пресвятой Богородицы", 44.201879, 43.125393),
        ("Городской парк культуры и отдыха", 44.210135, 43.130943),
        ("Музей Морской и Воинской Славы", 44.201032, 43.124730),
        ("Аллея на проспекте им. Карла Маркса", 44.209597, 43.136186),
        ("Музей писателя А. П. Бибика (Минераловодский краеведческий музей)", 44.202551, 43.121604),
        ("Святой источник Архангела Михаила", 44.166255, 43.109945),
        ("Терский Конный Завод", 44.144568, 43.087598)
    ],
    "Кафе": [
        ("Уголь", 44.211416, 43.140055),
        ("Roma", 44.198499, 43.123392),
        ("Сикварули", 44.199778, 43.133893),
        ("У Ламары", 44.204496, 43.106090),
        ("Палермо", 44.202279, 43.134549),
        ("Coffee sounds", 44.207475, 43.137091),
        ("Гиро", 44.211503, 43.139553),
        ("Caffe_ine", 44.210544, 43.140325),
        ("Вкусняшка", 44.211959, 43.139427),
        ("КирХан", 44.211590, 43.139202)
    ],
    "Отели": [
        ("Гостиница Прометей", 44.210641, 43.142301),
        ("Гостиница Оазис", 44.199668, 43.123868),
        ("Отель Кавказ", 44.210111, 43.133435),
        ("Гостевой дом София", 44.225028, 43.137396),
        ("Сити & Бизнес Отель", 44.211416, 43.140055),
        ("Отель Астэри", 44.197891, 43.124721)
    ],
    "Транспорт": [
        ("Аэропорт Минеральные воды", 44.217773, 43.087729),
        ("Ж/Д вокзал", 44.212753, 43.141017),
        ("Автовокзал Минеральные Воды", 44.208638, 43.121694),
        ("Автовокзал Мин. Воды Новый", 44.198783, 43.117975)
    ]
}

# Формируем GeoJSON
features = []
for cat, places in categories.items():
    for name, lat, lon in places:
        features.append({
            "type": "Feature",
            "properties": {
                "category": cat,
                "name": name
            },
            "geometry": {
                "type": "Point",
                "coordinates": [lon, lat]  # порядок: долгота, широта
            }
        })

geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Сохраняем в файл
with open("places.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("✅ Файл places.geojson успешно создан!")
from google.colab import files
files.download("places.geojson")

✅ Файл places.geojson успешно создан!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import json
import os
import folium
from folium import FeatureGroup, LayerControl
from folium.plugins import MarkerCluster, HeatMap, MiniMap, Fullscreen, MousePosition

# --- Настройки ---
OUT_DIR = "docs"
OUT_FILE = os.path.join(OUT_DIR, "index.html")
os.makedirs(OUT_DIR, exist_ok=True)

CENTER = (44.2167, 43.1333)
ZOOM_START = 13

COLORS = {
    "Достопримечательности": "darkred",
    "Кафе": "orange",
    "Отели": "darkblue",
    "Транспорт": "cadetblue",
}

# --- Загружаем GeoJSON ---
with open("places.geojson", "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Разбиваем по категориям ---
categories = {"Достопримечательности": [], "Кафе": [], "Отели": [], "Транспорт": []}
for feature in data["features"]:
    cat = feature["properties"]["category"]
    name = feature["properties"]["name"]
    lon, lat = feature["geometry"]["coordinates"]
    if cat in categories:
        categories[cat].append({"name": name, "lat": lat, "lon": lon})

# --- Функция для создания кастомного кластера ---
def create_cluster_group(name, color):
    from folium.plugins import MarkerCluster
    cluster = MarkerCluster(
        name=name,
        icon_create_function=f"""
        function(cluster) {{
            return new L.DivIcon({{
                html: '<div style="background-color:{color};'
                      + 'color:white;border-radius:50%;'
                      + 'width:40px;height:40px;'
                      + 'display:flex;align-items:center;justify-content:center;'
                      + 'font-weight:bold;border:2px solid white;">'
                      + cluster.getChildCount() + '</div>',
                className: 'marker-cluster'
            }});
        }}
        """
    )
    return cluster

# --- Создаём карту ---
m = folium.Map(location=CENTER, zoom_start=ZOOM_START, tiles="CartoDB positron", control_scale=True)
folium.TileLayer("OpenStreetMap").add_to(m)
MiniMap(toggle_display=True).add_to(m)
Fullscreen().add_to(m)
MousePosition(prefix="Коорд:").add_to(m)

# --- Создаём группы ---
fg_groups = {}
cluster_groups = {}

for cat, color in COLORS.items():
    fg = FeatureGroup(name=cat, show=True)
    cluster = create_cluster_group(cat, color)
    cluster.add_to(fg)
    fg.add_to(m)
    fg_groups[cat] = fg
    cluster_groups[cat] = cluster

# --- Добавляем точки ---
for cat, places in categories.items():
    for p in places:
        popup = f"<b>{p['name']}</b>"
        folium.Marker(
            [p["lat"], p["lon"]],
            popup=popup,
            tooltip=p["name"],
            icon=folium.Icon(color=COLORS[cat], icon="info-sign")
        ).add_to(cluster_groups[cat])

# --- HeatMap ---
fg_heat = FeatureGroup(name="Плотность интересных мест", show=False)
heat_data = [[p["lat"], p["lon"]] for plist in categories.values() for p in plist]
HeatMap(heat_data, radius=25, blur=18, min_opacity=0.3).add_to(fg_heat)
fg_heat.add_to(m)

# --- Легенда ---
legend_html = """
<div style='position: fixed; bottom: 30px; left: 10px;
z-index:9999; background-color: rgba(255,255,255,0.95);
border:2px solid #555; padding:10px; width:240px; font-size:14px;
border-radius:6px;'>
<h4 style="margin:0 0 6px 0">Туристическая карта</h4>
<span style="background:#8B0000;width:12px;height:12px;display:inline-block;margin-right:8px;"></span> Достопримечательности<br>
<span style="background:#FF8C00;width:12px;height:12px;display:inline-block;margin-right:8px;"></span> Кафе<br>
<span style="background:#00008B;width:12px;height:12px;display:inline-block;margin-right:8px;"></span> Отели<br>
<span style="background:#5F9EA0;width:12px;height:12px;display:inline-block;margin-right:8px;"></span> Транспорт<br>
<hr style="margin:8px 0;">
<small>Минеральные Воды © 2025</small>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# --- Контроль слоёв ---
LayerControl(collapsed=False).add_to(m)

# --- Сохраняем ---
m.save(OUT_FILE)
print(f"✅ Карта сохранена в {OUT_FILE}")

✅ Карта сохранена в docs/index.html
